检查内容：
+ 过拟合情况
+ 增加投票机制后的准确率
+ top5的情况
+ auc计算

In [1]:
#-*- coding:utf-8 -*-
import pandas as pd
from scipy.sparse import coo_matrix
import collections
import random
import time
import numpy as np
import tensorflow as tf
from data_input_fast_random import Data_set
from utils import *

#**************************************feed_dict***********************************************

def pull_all():
    #该地方插入函数，把query_iin，doc_positive_in,doc_negative_in转化成one_hot，再转化成coo_matrix
    query_in,doc_positive_in,doc_negative_in = train_data_set.get_one_hot_from_batch(query_BS)
    
    query_in = coo_matrix(query_in)
    doc_positive_in = coo_matrix(doc_positive_in)
    doc_negative_in = coo_matrix(doc_negative_in)
    
    query_in = tf.SparseTensorValue(
        np.transpose([np.array(query_in.row, dtype=np.int64), np.array(query_in.col, dtype=np.int64)]),
        np.array(query_in.data, dtype=np.float),
        np.array(query_in.shape, dtype=np.int64))
    doc_positive_in = tf.SparseTensorValue(
        np.transpose([np.array(doc_positive_in.row, dtype=np.int64), np.array(doc_positive_in.col, dtype=np.int64)]),
        np.array(doc_positive_in.data, dtype=np.float),
        np.array(doc_positive_in.shape, dtype=np.int64))
    doc_negative_in = tf.SparseTensorValue(
        np.transpose([np.array(doc_negative_in.row, dtype=np.int64), np.array(doc_negative_in.col, dtype=np.int64)]),
        np.array(doc_negative_in.data, dtype=np.float),
        np.array(doc_negative_in.shape, dtype=np.int64))

    return query_in, doc_positive_in, doc_negative_in


def feed_dict_train(on_training):
    """
    input: data_sets is a dict and the value type is numpy
    describe: to match the text classification the data_sets's content is the doc in df
    """
    query, doc_positive, doc_negative = pull_all()
        
    return {query_in: query, doc_positive_in: doc_positive, doc_negative_in: doc_negative,
            on_train: on_training}

def feed_dict_predict(sentence,doc_positive_spt,on_training=True):
    """
    input: data_sets is a dict and the value type is numpy
    describe: to match the text classification the data_sets's content is the doc in df
    """
    #该地方插入函数，把query_iin，doc_positive_in,doc_negative_in转化成one_hot，再转化成coo_matrix
    query = train_data_set.get_one_hot_from_sentence(sentence)
    
    query = coo_matrix(query)

    query = tf.SparseTensorValue(
        np.transpose([np.array(query.row, dtype=np.int64), np.array(query.col, dtype=np.int64)]),
        np.array(query.data, dtype=np.float),
        np.array(query.shape, dtype=np.int64))

    return {query_in: query, doc_positive_in: doc_positive_spt,on_train: on_training}

def feed_dict_triple(query,doc_pos,doc_neg,on_training=True):
    """
    input: data_sets is a dict and the value type is numpy
    describe: to match the text classification the data_sets's content is the doc in df
    """
    #该地方插入函数，把query_iin，doc_positive_in,doc_negative_in转化成one_hot，再转化成coo_matrix
    query = train_data_set.get_one_hot_from_sentence(query)
    doc_positive = train_data_set.get_one_hot_from_sentence(doc_pos)
    doc_negative = train_data_set.get_one_hot_from_sentence(doc_neg)
    
    query = coo_matrix(query)
    doc_positive = coo_matrix(doc_positive)
    doc_negative = coo_matrix(doc_negative)
    
    query = tf.SparseTensorValue(
        np.transpose([np.array(query.row, dtype=np.int64), np.array(query.col, dtype=np.int64)]),
        np.array(query.data, dtype=np.float),
        np.array(query.shape, dtype=np.int64))
    doc_positive = tf.SparseTensorValue(
        np.transpose([np.array(doc_positive.row, dtype=np.int64), np.array(doc_positive.col, dtype=np.int64)]),
        np.array(doc_positive.data, dtype=np.float),
        np.array(doc_positive.shape, dtype=np.int64))
    doc_negative = tf.SparseTensorValue(
        np.transpose([np.array(doc_negative.row, dtype=np.int64), np.array(doc_negative.col, dtype=np.int64)]),
        np.array(doc_negative.data, dtype=np.float),
        np.array(doc_negative.shape, dtype=np.int64))
    
    return {query_in: query, doc_positive_in: doc_positive, doc_negative_in: doc_negative,on_train: on_training}

def predict_label_n_with_sess(sess,sentence_list):
    result_list = []
    for i,sentence in enumerate(sentence_list):
        pred_prob_v,pred_label_v = sess.run([pred_prob,pred_label],feed_dict=feed_dict_predict(sentence,doc_main_question_spt))
        pred_main_question = train_data_set.get_main_question_from_label_index(pred_label_v)
        result_list.append(sentence + ":" +pred_main_question)
    return result_list

def evaluate_test_with_sess(sess,test_question_query_list,test_question_label_list):
    count = 0
    acc = 0
    for i,sentence in enumerate(test_question_query_list):
        pred_prob_v,pred_label_v = sess.run([pred_prob,pred_label],feed_dict=feed_dict_predict(sentence,doc_main_question_spt))
        pred_main_question = train_data_set.get_main_question_from_label_index(pred_label_v)
        if pred_main_question == test_question_label_list[i]:
            acc += 1
        count += 1
    return acc/float(count)




In [2]:
# the constant
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('summaries_dir', 'Summaries/', 'Summaries directory')
flags.DEFINE_string('train_write_name', 'train_fc2_300000_0.01', 'Summaries directory')
flags.DEFINE_string('checkpoint_name', 'model_fc2_300000_0.01.ckpt', 'Summaries directory')
flags.DEFINE_string('model_dir', 'model/', 'model directory')
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('step_num', 300000, 'batch_step')
flags.DEFINE_bool('gpu', 0, "Enable GPU or not")
flags.DEFINE_integer('print_cycle', 50000, "how many batches to print")

# the data_set and dataframe
train_data_set = Data_set(data_path='data/train_data.csv') #the train dataset
train_data_df = pd.read_csv('data/train_data.csv',encoding='utf-8')
train_question_query_list = list(train_data_df['query'])
train_question_label_list = list(train_data_df['main_question'])
test_data_df = pd.read_csv('data/main2one_0.1.csv',encoding='utf-8')
test_question_query_list = list(test_data_df['query'])
test_question_label_list = list(test_data_df['main_question'])

# coo fisrt
doc_main_question = train_data_set.get_one_hot_from_main_question()
doc_main_question = coo_matrix(doc_main_question)
doc_main_question_spt = tf.SparseTensorValue(
    np.transpose([np.array(doc_main_question.row, dtype=np.int64), np.array(doc_main_question.col, dtype=np.int64)]),
    np.array(doc_main_question.data, dtype=np.float),
    np.array(doc_main_question.shape, dtype=np.int64))

# the arg of triple-net
input_layer_num = train_data_set.get_word_num()
main_question_num =  train_data_set.get_main_question_num()
query_BS = 100

# the architecture of the triple-net
is_norm = False
layer1_len = 400
layer2_len = 120
layer3_len = 120

#input
query_in,doc_positive_in,doc_negative_in,on_train = input_layer(input_layer_num)
#fc1 
query_layer1_out,doc_pos_layer1_out,doc_neg_layer1_out = fc_layer(query_in,doc_positive_in,doc_negative_in,input_layer_num,layer1_len,'FC1',True,is_norm)
#fc2 
query_y,doc_positive_y,doc_negative_y = fc_layer(query_layer1_out,doc_pos_layer1_out,doc_neg_layer1_out,layer1_len,layer2_len,'FC2',False,is_norm)
#fc3
#query_y,doc_positive_y,doc_negative_y = fc_layer(query_layer2_out,doc_positive_layer2_out,doc_negative_layer2_out,layer2_len,layer3_len,'FC3',False,is_norm)
#loss
cos_sim,prob,loss = train_loss_layer(query_y,doc_positive_y,doc_negative_y,query_BS)
#acc
accuracy = accuracy_layer(prob)
#pred_label
pred_prob,pred_label = predict_layer(query_y,doc_positive_y,main_question_num)



read over,time:0.140769958496
word_dict_num:2923
build dict over,time:0.0769879817963


In [20]:
def evaluate_on_train():
    #检查训练集的acc
    saver = tf.train.Saver()

    #写一个函数查看输入query和输出类别
    config = tf.ConfigProto() 
    config.gpu_options.allow_growth = True
    if not FLAGS.gpu:
        config = tf.ConfigProto(device_count= {'GPU' : 0},allow_soft_placement=True)
    with tf.Session(config=config) as sess:     
        saver.restore(sess, FLAGS.model_dir+FLAGS.checkpoint_name)
        print "Model restored."
        count = 0
        acc = 0
        for i,sentence in enumerate(train_question_query_list):
            pred_prob_v,pred_label_v = sess.run([pred_prob,pred_label],feed_dict=feed_dict_predict(sentence,doc_main_question_spt))
            pred_main_question = train_data_set.get_main_question_from_label_index(pred_label_v)
            if pred_main_question == train_question_label_list[i]:
                acc += 1
            count += 1
    print acc/float(count)
    
def evaluate_on_test():
    #检查训练集的acc
    saver = tf.train.Saver()

    #写一个函数查看输入query和输出类别
    config = tf.ConfigProto() 
    config.gpu_options.allow_growth = True
    if not FLAGS.gpu:
        config = tf.ConfigProto(device_count= {'GPU' : 0},allow_soft_placement=True)
    with tf.Session(config=config) as sess:     
        saver.restore(sess, FLAGS.checkpoint_name)
        print "Model restored."
        count = 0
        acc = 0
        for i,sentence in enumerate(test_question_query_list):
            pred_prob_v,pred_label_v = sess.run([pred_prob,pred_label],feed_dict=feed_dict_predict(sentence,doc_main_question_spt))
            pred_main_question = train_data_set.get_main_question_from_label_index(pred_label_v)
            if pred_main_question == test_question_label_list[i]:
                acc += 1
            count += 1
    print acc/float(count)

def predict_label(sentence):
    #检查训练集的acc
    saver = tf.train.Saver()

    #写一个函数查看输入query和输出类别
    config = tf.ConfigProto() 
    config.gpu_options.allow_growth = True
    if not FLAGS.gpu:
        config = tf.ConfigProto(device_count= {'GPU' : 0},allow_soft_placement=True)
    with tf.Session(config=config) as sess:     
        saver.restore(sess, FLAGS.model_dir+FLAGS.checkpoint_name)
        print "Model restored."
        start = time.time()
        pred_prob_v,pred_label_v = sess.run([pred_prob,pred_label],feed_dict=feed_dict_predict(sentence,doc_main_question_spt))
        pred_main_question = train_data_set.get_main_question_from_label_index(pred_label_v)
        end = time.time()
        print end - start
        print sentence,pred_main_question

def get_badcase_test():
    #检查训练集的acc
    saver = tf.train.Saver()
    prob_list = []
    query_list = []
    doc_pos_list = []
    doc_pred_list = []
    #写一个函数查看输入query和输出类别
    config = tf.ConfigProto() 
    config.gpu_options.allow_growth = True
    if not FLAGS.gpu:
        config = tf.ConfigProto(device_count= {'GPU' : 0},allow_soft_placement=True)
    with tf.Session(config=config) as sess:     
        saver.restore(sess,FLAGS.model_dir+FLAGS.checkpoint_name)
        print "Model restored."
        for i,sentence in enumerate(test_question_query_list):
            if type(sentence) != float:
                pred_prob_v,pred_label_v = sess.run([pred_prob,pred_label],feed_dict=feed_dict_predict(sentence,doc_main_question_spt))
                pred_main_question = train_data_set.get_main_question_from_label_index(pred_label_v)
                if pred_main_question != test_question_label_list[i]:
                    prob_list.append(pred_prob_v)
                    query_list.append(sentence)
                    doc_pos_list.append( test_question_label_list[i])
                    doc_pred_list.append(pred_main_question)
    return prob_list,query_list,doc_pos_list,doc_pred_list

In [5]:
evaluate_on_test()

INFO:tensorflow:Restoring parameters from model_fc2_300000_0.01.ckpt


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for model_fc2_300000_0.01.ckpt
	 [[Node: save_1/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_1/Const_0_0, save_1/RestoreV2/tensor_names, save_1/RestoreV2/shape_and_slices)]]

Caused by op u'save_1/RestoreV2', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/usr/local/lib/python2.7/dist-packages/tornado/iolo

In [7]:
evaluate_on_train()

INFO:tensorflow:Restoring parameters from model/model_test
Model restored.
0.481142608175


In [21]:
xx = get_badcase_test()

INFO:tensorflow:Restoring parameters from model/model_fc2_300000_0.01.ckpt
Model restored.


In [22]:
prob_list,query_list,doc_pos_list,doc_pred_list = xx

In [24]:
new_df = pd.DataFrame(data={'query':query_list,'main_question':doc_pos_list,'pred_question':doc_pred_list})

In [26]:
new_df.to_csv('zfy_result.csv',encoding='utf-8')

In [28]:
new_df.head(100)

,main_question,pred_question,query
0,开通金卡有什么优惠,会员卡规则,会员卡规则
1,你好·······,你好,你好
2,但你们没有给我运费,退货退款完成了 需要退货运费,退货退款完成了，需要退货运费
3,订单券并未收到,补偿的现金券没有收到,补偿的现金券没有收到
4,50元券在哪里,在哪里查看现金券,在哪里查看现金券
5,是什么时候可以发货,发货时间,发货时间
6,年前送达就好,春节期间具体发货和物流时效调整,春节期间具体发货和物流时效调整
7,衣服尺码偏小吗,尺码是否标准,衣服尺码标准吗
8,没办法查到货到哪里了吗,查询物流走件信息,查询物流走件信息
9,用大码可以吧 如果买你这个尿片,选择合适的内裤,如何选择纸尿裤
